In [1]:
from imgaug import augmenters as iaa
import segmentation_models as sm

from tensorflow import keras
keras.backend.set_image_data_format('channels_last')



from segmentation_models import Unet
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model


from skimage import io


#import imgaug as ia


Segmentation Models: using `tf.keras` framework.


In [2]:
model = sm.Unet(backbone_name=None)

ValueError: No such model `None`, available models: ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'seresnet18', 'seresnet34', 'seresnet50', 'seresnet101', 'seresnet152', 'seresnext50', 'seresnext101', 'senet154', 'resnext50', 'resnext101', 'vgg16', 'vgg19', 'densenet121', 'densenet169', 'densenet201', 'inceptionresnetv2', 'inceptionv3', 'mobilenet', 'mobilenetv2', 'efficientnetb0', 'efficientnetb1', 'efficientnetb2', 'efficientnetb3', 'efficientnetb4', 'efficientnetb5', 'efficientnetb6', 'efficientnetb7']

In [14]:
import glob
import numpy as np
import os

In [8]:
#BACKBONE = 'vgg16'
BATCH_SIZE = 8
#CLASSES = ['car']
#LR = 0.001
EPOCHS = 500

In [4]:
preprocess_input = sm.get_preprocessing(BACKBONE)

NameError: name 'BACKBONE' is not defined

In [30]:
# define network parameters
n_classes = 2  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=2, activation='softmax',input_shape=(256,256,7))

NameError: name 'BACKBONE' is not defined

In [ ]:
#custom input

In [4]:
#del base_model
del  model

NameError: name 'model' is not defined

In [9]:
base_model = Unet(
    backbone_name='vgg16', 
    encoder_weights='./files/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
    activation='sigmoid'
)

inp = Input(shape=(256, 256, 16))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = base_model(l1)

model = Model(inp, out, name=base_model.name)

# small update 29.11.2020 Misha has noticed that it is possible to 
# set input shape in model creation too
#model = Unet(
#    backbone_name='vgg16', 
#    encoder_weights=None,
#    activation='sigmoid',
#    input_shape=(256, 256, 5)
#)

# continue with usual steps: compile, fit, etc.

* LR Utils

In [2]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-5
    if epoch == 180:
        lr *= 0.5e-3
    elif epoch == 160:
        lr *= 1e-3
    elif epoch == 120:
        lr *= 1e-2
    elif epoch == 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [10]:
lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)

lr_reducer = keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-8)

------

In [12]:
#optim = keras.optimizers.Adam(learning_rate=lr_schedule(0))
optim = keras.optimizers.Adam()

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
#focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
focal_loss = sm.losses.BinaryFocalLoss() 
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [17]:
# folder where dates is intermidiate in season
# with show -> without snow
# without show -> with snow
"""flds = [
    'T39VVJ_pair_0.tif',
    'T39VWH_pair_2.tif',
    'T39VVJ_pair_4.tif',
    'T40VEM_pair_0.tif',
    'T39VWJ_pair_5.tif',
    'T39VXJ_pair_3.tif',
    'T39VWJ_pair_3.tif',
    'T39VXH_pair_5.tif',
    'T39VXH_pair_3.tif',
    'T39VXJ_pair_2.tif',
    'T39VXJ_pair_5.tif',
    'T40VDJ_pair_1.tif',
    'T40VEL_pair_4.tif',
    'T40VEL_pair_1.tif',
    'T40VDJ_pair_4.tif',
    'T39VWH_pair_4.tif',
    'T40VEM_pair_2.tif',
    'T40VFK_pair_8.tif',
    'T40VFK_pair_3.tif',
    'T39VWH_pair_8.tif',
    'T41VLE_pair_3.tif',
    'T40VFK_pair_5.tif',
    'T41VLE_pair_1.tif',
    'T41VLE_pair_2.tif'
]"""

# transition period filtered (with high and middle confidence)
flds = [
    'T39VVJ_pair_0.tif',
    'T39VWH_pair_2.tif',
    'T40VEM_pair_0.tif',
    'T39VXJ_pair_3.tif',
    'T39VXH_pair_5.tif',
    'T39VXH_pair_3.tif',
    'T39VXJ_pair_2.tif',
    'T39VXJ_pair_5.tif',
    'T40VDJ_pair_1.tif',
    'T40VEL_pair_1.tif',
    'T40VDJ_pair_4.tif',
    'T39VWH_pair_4.tif',
    'T40VEM_pair_2.tif',
    'T39VWH_pair_8.tif',
    'T41VLE_pair_3.tif',
    'T40VFK_pair_5.tif',
    'T41VLE_pair_1.tif',
    'T41VLE_pair_2.tif',
]


#summer folders
#without snow -> without snow
'''flds = [
     'T39VVJ_pair_1.tif',
     'T39VVJ_pair_2.tif',
     'T39VVJ_pair_3.tif',
     'T39VWH_pair_3.tif',
     'T39VXH_pair_0.tif',
     'T39VWJ_pair_0.tif',
     'T39VWH_pair_6.tif',
     'T39VWH_pair_7.tif',
     'T39VWJ_pair_4.tif',
     'T39VWJ_pair_7.tif',
     'T39VWJ_pair_8.tif',
     'T39VXH_pair_4.tif',
     'T39VXH_pair_6.tif',
     'T39VXH_pair_7.tif',
     'T39VXJ_pair_6.tif',
     'T40VDJ_pair_2.tif',
     'T40VDJ_pair_3.tif',
     'T40VEM_pair_4.tif',
     'T40VEL_pair_0.tif',
     'T40VEM_pair_3.tif',
     'T40VFK_pair_1.tif',
     'T40VFK_pair_2.tif',
     'T40VFK_pair_6.tif',
     'T40VFK_pair_7.tif',
     'T41VLE_pair_0.tif',
]'''

# summer changes filtered (with high confidence)
'''flds = [
    'T39VWH_pair_3.tif',
    'T39VXH_pair_0.tif',
    'T39VWH_pair_7.tif',
    'T39VXH_pair_6.tif',
    'T39VXH_pair_7.tif',
    'T39VXJ_pair_6.tif',
    'T40VDJ_pair_2.tif',
    'T40VDJ_pair_3.tif',
    'T40VEM_pair_4.tif',
    'T41VLE_pair_0.tif',
]'''



#winter changes
#with snow -> with snow
"""flds = [
     'T40VEK_pair_0.tif',
     'T40VEK_pair_1.tif',
     'T40VEK_pair_2.tif',
     'T40VEK_pair_4.tif',
     'T40VEM_pair_1.tif',
     'T40VEK_pair_3.tif',
     'T40VDJ_pair_0.tif',
     'T40VFK_pair_4.tif',
     'T39VXJ_pair_0.tif',
     'T39VWJ_pair_6.tif',
     'T39VXH_pair_1.tif',
     'T40VFK_pair_0.tif',
     'T40VEM_pair_5.tif',
     'T39VWH_pair_0.tif',
     'T39VXJ_pair_1.tif',
     'T39VXH_pair_2.tif',
     'T39VWJ_pair_1.tif',
     'T40VEL_pair_2.tif',
     'T40VEM_pair_6.tif',
     'T40VEL_pair_3.tif',
     'T39VWH_pair_1.tif',
     'T39VXJ_pair_4.tif',
     'T39VWJ_pair_2.tif',
     'T40VEL_pair_5.tif',
     'T39VWH_pair_5.tif',
     'T40VEL_pair_6.tif',
]"""


#get all tifs from train folder
#allTifs = list(glob.glob('../../ds/tiles_256_256_27_add/*/*.tif'))

# with object tiffs
#allTifs = list(glob.glob('../../ds/tiles_256_256_27/*/*with_object.tif'))

# get all tiffs from defined list
#allTifs = []
#for fldName in flds:
#    allTifs.extend(list(glob.glob(f'../../ds/tiles_256_256_27/{fldName}/*.tif')))

#add tiles + checked tiles 
# delete control data
notControl = ['T40VDN', 'T40VEN', 'T39VUH', 'T39VUK']
addTiles = []
for tile in list(glob.glob(f'../../ds/tiles_256_256_27_add/*/*with_object.tif')):
    tileName = os.path.basename(tile)
    #print(tileName[:6])
    if tileName[:6] in notControl:
        addTiles.append(tile)
allTifs = (
    list(glob.glob(f'../../ds/tiles_256_256_27_after_check/*/*.tif'))
    + addTiles
)

#shuffle
np.random.shuffle(allTifs)

#validate 15%
#validate by hist by month
validSize = int(len(allTifs)*0.15)
    
train_generator = DataGenerator(allTifs[:-validSize],batch_size=BATCH_SIZE)
valid_generator = DataGenerator(allTifs[-validSize:],batch_size=BATCH_SIZE)

In [18]:
callbacks = [
    keras.callbacks.ModelCheckpoint('./files/BackBoneTest.h5', 
                                    save_weights_only=True, 
                                    save_best_only=True, 
                                    mode='min'),
    #keras.callbacks.ReduceLROnPlateau(),
    #lr_scheduler,
    lr_reducer
]

In [19]:
history = model.fit(
    train_generator, 
    steps_per_epoch=len(train_generator), 
    epochs=EPOCHS, 
    callbacks=callbacks, 
    validation_data=valid_generator, 
    validation_steps=len(valid_generator),
)

Epoch 1/500
319/319 [==============================] - 414s 1s/step - loss: 0.8978 - iou_score: 0.1016 - f1-score: 0.1655 - val_loss: 1.0317 - val_iou_score: 0.0179 - val_f1-score: 0.0179 - lr: 0.0010
Epoch 2/500
319/319 [==============================] - 406s 1s/step - loss: 0.7726 - iou_score: 0.1675 - f1-score: 0.2664 - val_loss: 0.9397 - val_iou_score: 0.0513 - val_f1-score: 0.0859 - lr: 0.0010
Epoch 3/500
319/319 [==============================] - 413s 1s/step - loss: 0.7310 - iou_score: 0.1981 - f1-score: 0.3063 - val_loss: 1.0220 - val_iou_score: 4.3331e-09 - val_f1-score: 4.3331e-09 - lr: 0.0010
Epoch 4/500
319/319 [==============================] - 408s 1s/step - loss: 0.7043 - iou_score: 0.2161 - f1-score: 0.3322 - val_loss: 1.0307 - val_iou_score: 0.0179 - val_f1-score: 0.0179 - lr: 0.0010
Epoch 5/500
319/319 [==============================] - 398s 1s/step - loss: 0.6557 - iou_score: 0.2517 - f1-score: 0.3794 - val_loss: 1.0252 - val_iou_score: 0.0357 - val_f1-score: 0.0357 

Epoch 41/500
319/319 [==============================] - 399s 1s/step - loss: 0.4645 - iou_score: 0.4158 - f1-score: 0.5586 - val_loss: 0.4713 - val_iou_score: 0.4045 - val_f1-score: 0.5474 - lr: 1.0000e-05
Epoch 42/500
319/319 [==============================] - 396s 1s/step - loss: 0.4641 - iou_score: 0.4195 - f1-score: 0.5593 - val_loss: 0.4956 - val_iou_score: 0.3911 - val_f1-score: 0.5247 - lr: 3.1623e-06
Epoch 43/500
319/319 [==============================] - 392s 1s/step - loss: 0.4690 - iou_score: 0.4130 - f1-score: 0.5540 - val_loss: 0.4794 - val_iou_score: 0.4047 - val_f1-score: 0.5402 - lr: 3.1623e-06
Epoch 44/500
319/319 [==============================] - 400s 1s/step - loss: 0.4616 - iou_score: 0.4180 - f1-score: 0.5620 - val_loss: 0.4618 - val_iou_score: 0.4143 - val_f1-score: 0.5583 - lr: 3.1623e-06
Epoch 45/500
319/319 [==============================] - 397s 1s/step - loss: 0.4584 - iou_score: 0.4209 - f1-score: 0.5655 - val_loss: 0.4840 - val_iou_score: 0.4002 - val_f1-s

Epoch 81/500
319/319 [==============================] - 404s 1s/step - loss: 0.4617 - iou_score: 0.4190 - f1-score: 0.5621 - val_loss: 0.4539 - val_iou_score: 0.4174 - val_f1-score: 0.5664 - lr: 1.0000e-08
Epoch 82/500
319/319 [==============================] - 407s 1s/step - loss: 0.4641 - iou_score: 0.4176 - f1-score: 0.5596 - val_loss: 0.4623 - val_iou_score: 0.4189 - val_f1-score: 0.5577 - lr: 1.0000e-08
Epoch 83/500
319/319 [==============================] - 397s 1s/step - loss: 0.4603 - iou_score: 0.4194 - f1-score: 0.5632 - val_loss: 0.4631 - val_iou_score: 0.4134 - val_f1-score: 0.5566 - lr: 1.0000e-08
Epoch 84/500
319/319 [==============================] - 409s 1s/step - loss: 0.4507 - iou_score: 0.4279 - f1-score: 0.5728 - val_loss: 0.4781 - val_iou_score: 0.4094 - val_f1-score: 0.5404 - lr: 5.0000e-09
Epoch 85/500
319/319 [==============================] - 404s 1s/step - loss: 0.4683 - iou_score: 0.4128 - f1-score: 0.5551 - val_loss: 0.4713 - val_iou_score: 0.4284 - val_f1-s

Epoch 121/500
319/319 [==============================] - 402s 1s/step - loss: 0.4466 - iou_score: 0.4345 - f1-score: 0.5772 - val_loss: 0.4751 - val_iou_score: 0.4081 - val_f1-score: 0.5449 - lr: 5.0000e-09
Epoch 122/500
319/319 [==============================] - 406s 1s/step - loss: 0.4571 - iou_score: 0.4214 - f1-score: 0.5669 - val_loss: 0.4745 - val_iou_score: 0.4010 - val_f1-score: 0.5442 - lr: 5.0000e-09
Epoch 123/500
319/319 [==============================] - 399s 1s/step - loss: 0.4611 - iou_score: 0.4177 - f1-score: 0.5619 - val_loss: 0.4608 - val_iou_score: 0.4190 - val_f1-score: 0.5591 - lr: 5.0000e-09
Epoch 124/500
319/319 [==============================] - 408s 1s/step - loss: 0.4698 - iou_score: 0.4165 - f1-score: 0.5564 - val_loss: 0.4622 - val_iou_score: 0.4193 - val_f1-score: 0.5583 - lr: 5.0000e-09
Epoch 125/500
319/319 [==============================] - 412s 1s/step - loss: 0.4580 - iou_score: 0.4223 - f1-score: 0.5657 - val_loss: 0.4653 - val_iou_score: 0.4148 - val

KeyboardInterrupt: 

In [135]:
import pandas as pd
import os
modelPath = './files/add_SMSummerSeason_32.h5'
if os.path.exists(modelPath):
    raise Exception("File already exists")
else:
    model.save(modelPath)
    hist_df = pd.DataFrame(history.history) 
    hist_df.to_csv(modelPath.replace('.h5','.csv'))

# Custom data generator

In [16]:
class DataGenerator(keras.utils.Sequence):
    """
    Custom data generator for Unet PermForestChange
    
    Main parameter is unputTiffs - list to path to tiff files
    
    Sample usage 
    allTifs = list(glob.glob('../../ds/tiles_256_256_27/*/*.tif'))

    validSize = 250
    
    train_generator = DataGenerator(allTifs[:-validSize],batch_size=batch)
    
    """
    def __init__(self, inputTiffs, batch_size=32, num_classes=2, shuffle=True):
        self.inputTiffs = inputTiffs
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        #custom augmentation with imgaug lib
        self.aug_pipe = iaa.Sequential(
            [
                iaa.SomeOf((0,2),
                [
                   #iaa.Add((-30, -10)), # change brightness of images (by -10 to 10 of original value)
                   #iaa.Multiply((0.8, 1.2)),
                   
                   # Geometric transfroms
                    iaa.Rotate((-45,45)),
                    iaa.TranslateX(px=(-20, 20)),
                    iaa.TranslateY(px=(-20, 20)),
                ]
                ),
            ],
           random_order=True
       )

    def __len__(self):
        return len(self.inputTiffs) // self.batch_size

    def __getitem__(self, index):
        indexes = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        #print (indexes)
        batchTifs = [self.inputTiffs[k] for k in indexes]
        
        X, y = self._get_data(batchTifs)
        return X, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.inputTiffs))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def _get_data(self, batch):
        #X = inArray
        #y = # logic
        
        X = []
        y = []
        
        for idx,imagePath in enumerate(batch):
            
            #print (image)
            
            # raw image
            img = (io.imread(imagePath)/65536).astype('float32')
            
            # flip with cv2
            flip = np.random.binomial(1, .5)
            if flip > 0.5:
                #img = self.flip(img)
                img = self.aug_pipe.augment_image(img)
            
            # aug with imgaug lib
            #img = self.aug_pipe.augment_image(img)
            
            # x with features
            # channel indexes
            # mask - 0 
            # NEW
            #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
            #[ 1,     2,    3,    4,    5,    6,    7,    8,    9,    10,   11,   12,   13  ]
            # OLD
            #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
            #[ 14,    15,   16,   17,   18,   19,   20,   21,   22,   23,   24,   25,   26  ]
            X.append(np.array(
                np.dstack([
                    #img[:,:,17],# old red
                    #img[:,:,21], # new red
                    #img[:,:,25], # new red
                    #img[:,:,26], # new red
                    
                    # separate new
                    img[:,:,4],
                    #img[:,:,5],
                    #img[:,:,6],
                    #img[:,:,7],
                    img[:,:,8],
                    #img[:,:,13],
                    # separate old
                    img[:,:,17],
                    #img[:,:,18],
                    #img[:,:,19],
                    #img[:,:,20],
                    img[:,:,21],
                    #img[:,:,26],
                    #
                    img[:,:,4] - img[:,:,17], # dif red
                    img[:,:,17] - img[:,:,4], # dif for transitionChange
                    #img[:,:,8],
                    #img[:,:,21],
                    #img[:,:,13],
                    img[:,:,8] - img[:,:,21],
                    img[:,:,21] - img[:,:,8],
                    #(
                    #    (img[:,:,8] - img[:,:,4]) /((img[:,:,8] + img[:,:,4])+ 0.0001)- # new ndvi
                    #    (img[:,:,21] - img[:,:,17])/((img[:,:,21] + img[:,:,17])+0.0001) # old ndvi
                    #), # ndvi diff
                    #swvi dif
                    #(
                    #    (img[:,:,8] - img[:,:,13]) /((img[:,:,8] + img[:,:,13])+0.0001)-   # new swir
                    #    (img[:,:,21] - img[:,:,26])/((img[:,:,21] + img[:,:,26])+0.0001)  # old swir
                        
                    #),
                    #swvi dif 
                    #(
                    #    (img[:,:,21] - img[:,:,26])/((img[:,:,21] + img[:,:,26])+0.0001) -
                    #    (img[:,:,8] - img[:,:,13]) /((img[:,:,8] + img[:,:,13])+0.0001)   # new swir
                         # old swir
                        
                    #),
                    
                    img[:,:,13],
                    img[:,:,26],
                    img[:,:,13] - img[:,:,26],
                    img[:,:,26] - img[:,:,13], # dif for transitionChange
                    img[:,:,12], # dif for transitionChange
                    img[:,:,25], # dif for transitionChange
                    img[:,:,12] - img[:,:,25], # dif for transitionChange
                    img[:,:,25] - img[:,:,12], # dif for transitionChange
                    #(get_value_by_extent(imagePath))/100.0
                    
                ])).astype('float32'))
            
            # to one class all changes
            y.append(np.where(img[:,:,0]>0,1,0).astype('int'))
            
        # return x as preprocess With Backbone 
        #return preprocess_input(np.array(X)), np.array(y)
        
        return np.array(X), np.array(y)
    
    def flip(self,image):
        """
        Random flip image around x, y or both axes
        image is np.array
        """
        flipDirection = np.random.randint(-1,1,1)[0]
        return cv2.flip(image,flipDirection)
        
        

# MultiClass generator

In [1]:
#import imgaug as ia
from imgaug import augmenters as iaa

class DataGenerator(tf.keras.utils.Sequence):
    """
    Custom data generator for Unet PermForestChange
    
    Main parameter is unputTiffs - list to path to tiff files
    
    Sample usage 
    allTifs = list(glob.glob('../../ds/tiles_256_256_27/*/*.tif'))

    validSize = 250
    
    train_generator = DataGenerator(allTifs[:-validSize],batch_size=batch)
    
    """
    def __init__(self, inputTiffs, batch_size=32, num_classes=2, shuffle=True):
        self.inputTiffs = inputTiffs
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        #custom augmentation with imgaug lib
        self.aug_pipe = iaa.Sequential(
            [
                iaa.SomeOf((0,2),
                [
                    #iaa.Add((-30, -10)), # change brightness of images (by -10 to 10 of original value)
                    #iaa.Multiply((0.8, 1.2)),
                    
                    # Geometric transfroms
                    iaa.Rotate((-45,45)),
                    iaa.TranslateX(px=(-20, 20)),
                    iaa.TranslateY(px=(-20, 20)),
                ]
                ),
            ],
            random_order=True
        )

    def __len__(self):
        return len(self.inputTiffs) // self.batch_size

    def __getitem__(self, index):
        indexes = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        #print (indexes)
        batchTifs = [self.inputTiffs[k] for k in indexes]
        
        X, y = self._get_data(batchTifs)
        return X, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.inputTiffs))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def _get_data(self, batch):
        #X = inArray
        #y = # logic
        
        X = []
        y = []
        
        for idx,imagePath in enumerate(batch):
            
            #print (image)
            
            # raw image
            img = io.imread(imagePath)
            mask = img[:,:,0]
            img = (img/65536).astype('float32')
            
            # flip with cv2
            #flip = np.random.binomial(1, .5)
            #if flip > 0.5:
                #img = self.flip(img)
            
            # aug with imgaug lib
            #img = self.aug_pipe.augment_image(img)
            
            # x with features
            # channel indexes
            # mask - 0 
            # NEW
            #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
            #[ 1,     2,    3,    4,    5,    6,    7,    8,    9,    10,   11,   12,   13  ]
            # OLD
            #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
            #[ 14,    15,   16,   17,   18,   19,   20,   21,   22,   23,   24,   25,   26  ]
            X.append(np.array(
                np.dstack([
#img[:,:,17],# old red
                    #img[:,:,21], # new red
                    #img[:,:,25], # new red
                    #img[:,:,26], # new red
                    
                    # separate new
                    img[:,:,4],
                    #img[:,:,5],
                    #img[:,:,6],
                    #img[:,:,7],
                    img[:,:,8],
                    #img[:,:,13],
                    # separate old
                    img[:,:,17],
                    #img[:,:,18],
                    #img[:,:,19],
                    #img[:,:,20],
                    img[:,:,21],
                    #img[:,:,26],
                    #
                    img[:,:,4] - img[:,:,17], # dif red
                    img[:,:,17] - img[:,:,4], # dif for transitionChange
                    #img[:,:,8],
                    #img[:,:,21],
                    #img[:,:,13],
                    img[:,:,8] - img[:,:,21],
                    img[:,:,21] - img[:,:,8],
                    #(
                    #    (img[:,:,8] - img[:,:,4]) /((img[:,:,8] + img[:,:,4])+ 0.0001)- # new ndvi
                    #    (img[:,:,21] - img[:,:,17])/((img[:,:,21] + img[:,:,17])+0.0001) # old ndvi
                    #), # ndvi diff
                    #swvi dif
                    #(
                    #    (img[:,:,8] - img[:,:,13]) /((img[:,:,8] + img[:,:,13])+0.0001)-   # new swir
                    #    (img[:,:,21] - img[:,:,26])/((img[:,:,21] + img[:,:,26])+0.0001)  # old swir
                        
                    #),
                    #swvi dif 
                    #(
                    #    (img[:,:,21] - img[:,:,26])/((img[:,:,21] + img[:,:,26])+0.0001) -
                    #    (img[:,:,8] - img[:,:,13]) /((img[:,:,8] + img[:,:,13])+0.0001)   # new swir
                         # old swir
                        
                    #),
                    
                    img[:,:,13],
                    img[:,:,26],
                    img[:,:,13] - img[:,:,26],
                    img[:,:,26] - img[:,:,13], # dif for transitionChange
                    img[:,:,12], # dif for transitionChange
                    img[:,:,25], # dif for transitionChange
                    img[:,:,12] - img[:,:,25], # dif for transitionChange
                    img[:,:,25] - img[:,:,12], # dif for transitionChange
                    #(get_value_by_extent(imagePath))/100.0
                    
                    
                ])).astype('float32'))
            
            # to one class all changes
            #y.append(np.where(img[:,:,0]>0,1,0).astype('int'))
            y.append(mask)
            
        return np.array(X), np.array(y)
    
    def flip(self,image):
        """
        Random flip image around x, y or both axes
        image is np.array
        """
        flipDirection = np.random.randint(-1,1,1)[0]
        #return cv2.flip(image,flipDirection)
        
        

NameError: name 'tf' is not defined

In [112]:
import gdal
import os


def getRasterExtent (rasterPath):
    
    """
    Get raster extent
    """
    
    infoOptions = gdal.InfoOptions(format='json')
    infoResult = gdal.Info(rasterPath,options=infoOptions)
    # return as -te <xmin> <ymin> <xmax> <ymax>
    return ((infoResult["cornerCoordinates"]['lowerLeft'])
        +(infoResult["cornerCoordinates"]['upperRight']))

def get_value_by_extent(in_tile):
    
    """Get forest mask from another raster by extent"""
    
    tileSize = 256
    tileExtent = getRasterExtent(in_tile)
    tileID = os.path.basename(in_tile).split("_")[0]
    maskRaster = f"./addData/treeCover/{tileID}.tif"
    #print(in_tile)
    #print(maskRaster)
    fullExtent = getRasterExtent(maskRaster)
                                 
    tileXTopPixel = (tileExtent[0] - fullExtent[0])//10 - 1   
    tileYTopPixel = (fullExtent[3] - tileExtent[3])//10 - 1                             
                                 
    tr_opts = gdal.TranslateOptions(
        format="GTiff", 
        srcWin=[tileXTopPixel ,tileYTopPixel, tileSize, tileSize],
        outputType=gdal.GDT_Byte,
        outputBounds=[tileExtent[0],tileExtent[3],tileExtent[2],tileExtent[1]]
    )
    #print([extent[0],extent[3],extent[2], extent[1]])
    gdal.Translate('./addData/tempClip/temp.tif', maskRaster,options=tr_opts)
    arr = io.imread('./addData/tempClip/temp.tif')
    os.remove('./addData/tempClip/temp.tif')
    return arr

In [65]:
arr = get_value_by_extent(
    '../../ds/tiles_256_256_27_add/T40VDN_pair_0_2018_07_23_2018_09_24.tif/T40VDN_pair_0_2018_07_23_2018_09_24.tif_1877_10473_no_object.tif')/100.0
arr.shape

../../ds/tiles_256_256_27_add/T40VDN_pair_0_2018_07_23_2018_09_24.tif/T40VDN_pair_0_2018_07_23_2018_09_24.tif_1877_10473_no_object.tif


(256, 256)